## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [1]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/ozge/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [2]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to the Amazon Comprehend API, and uses it to score a single review:

* [Documentation - Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment - if this happens, you will need to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [3]:
import boto3
client =  boto3.client('comprehend',region_name='eu-central-1',aws_access_key_id='',aws_secret_access_key='')

In [22]:
text = """My order was delayed by several days without any updates or communication from the seller. Terrible shipping service."""

client.detect_sentiment(Text=text, LanguageCode='en')

{'Sentiment': 'NEGATIVE',
 'SentimentScore': {'Positive': 0.00015661792713217437,
  'Negative': 0.9997037053108215,
  'Neutral': 0.00011433633335400373,
  'Mixed': 2.5389967049704865e-05},
 'ResponseMetadata': {'RequestId': '11588c6a-d649-408c-8fe5-58c52c7055dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11588c6a-d649-408c-8fe5-58c52c7055dc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '168',
   'date': 'Thu, 25 Jan 2024 11:52:13 GMT'},
  'RetryAttempts': 0}}

In [17]:
def score_review(review):
    ret_dict=client.detect_sentiment(Text=review[:5000], LanguageCode='en')
    
    if (ret_dict['Sentiment']=="NEGATIVE"):
        return 'neg'
    elif (ret_dict['Sentiment']=="POSITIVE"):
        return 'pos'
    elif (ret_dict['SentimentScore']['Positive']> ret_dict['SentimentScore']['Negative']):
        return 'pos'
    else:
        return 'neg'
    

### Score each review

Now, we can use the function you defined to score each of the reviews

In [20]:
# Create 2 smaller subsets for testing
subset_pos = reviews_pos[:10]
subset_neg = reviews_neg[:10]

results_pos = []
# When comfortable with results switch `subset_pos` to reviews_pos`
for review in reviews_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
# When comfortable with results switch `subset_neg` to reviews_neg`
for review in reviews_neg:
    result = score_review(review)
    results_neg.append(result)

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [21]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 64.7% correct
Negative reviews: 68.0% correct
Overall accuracy: 66.35% correct
